# 1. load the keypoints to fiftyone

In [ ]:
import fiftyone as fo
import fiftyone.utils as fou
import os

# Assuming you have a list of image paths and corresponding TXT file paths
image_paths = [...]  # List of image file paths
txt_paths = [...]  # List of corresponding TXT file paths for each image

# Create a FiftyOne dataset
dataset = fo.Dataset("prawn_pose_estimation")
def parse_pose_estimation(txt_file):
    """
    Parse the pose estimation data from a TXT file.

    Parameters:
    txt_file (str): Path to the TXT file.

    Returns:
    list: A list of pose estimation data (each line parsed into a list of values).
    """
    pose_estimations = []
    with open(txt_file, 'r') as f:
        for line in f:
            pose_estimations.append([float(x) for x in line.strip().split()])
    return pose_estimations

# Loop over images and corresponding TXT files
for img_path, txt_path in zip(image_paths, txt_paths):
    # Parse the pose estimation data from the TXT file
    pose_estimations = parse_pose_estimation(txt_path)
    
    # List to hold all detections and keypoints for the current image
    detections = []
    
    for pose in pose_estimations:
        # Extract and scale bounding box and keypoints
        x_center_scaled = pose[1] * 5312
        y_center_scaled = pose[2] * 2988
        width_scaled = pose[3] * 5312
        height_scaled = pose[4] * 2988

        keypoints = []
        for i in range(5, len(pose), 3):
            x_kp_scaled = pose[i] * 5312
            y_kp_scaled = pose[i + 1] * 2988
            confidence = pose[i + 2]
            keypoints.append(fo.Keypoint(point=[x_kp_scaled, y_kp_scaled], confidence=confidence))

        bounding_box = [
            (x_center_scaled - width_scaled / 2) / 5312,
            (y_center_scaled - height_scaled / 2) / 2988,
            width_scaled / 5312,
            height_scaled / 2988
        ]

        # Create a detection object
        detection = fo.Detection(label="prawn", bounding_box=bounding_box, keypoints=keypoints)
        detections.append(detection)

    # Create a sample for FiftyOne
    sample = fo.Sample(filepath=img_path)
    sample["detections"] = fo.Detections(detections=detections)
    dataset.add_sample(sample)

# Launch the FiftyOne app to visualize
session = fo.launch_app(dataset)




# 2. load metadata from excel file

In [ ]:
import pandas as pd
import fiftyone as fo

# Read the Excel file
file_path = 'your_file_path.xlsx'  # Replace with your actual file path
metadata_df = pd.read_excel(file_path)

# Display the first few rows to ensure it's loaded correctly
print(metadata_df.head())


# Assuming you have a list of image paths
image_paths = [...]  # List of image file paths

# Create a FiftyOne dataset
dataset = fo.Dataset("prawn_metadata")

# Loop through each image and add metadata
for img_path in image_paths:
    # Extract the filename without extension
    filename = img_path.split('/')[-1].replace('.jpg', '')  # Adjust if not .jpg
    
    # Match the filename with the metadata
    metadata_row = metadata_df[metadata_df['file name'] == filename]
    
    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict()
        
        # Create a sample and attach metadata
        sample = fo.Sample(filepath=img_path)
        
        # Add each metadata field to the sample
        for key, value in metadata.items():
            if key != 'file name':  # Exclude the file name itself
                sample[key] = value
        
        # Add the sample to the dataset
        dataset.add_sample(sample)

# Launch the FiftyOne app to visualize
session = fo.launch_app(dataset)


# 3.bounding rectangle between detection and imagej

In [ ]:
import pandas as pd

# Load ImageJ bounding rectangles from a CSV file
imagej_bboxes_df = pd.read_csv('imagej_bboxes.csv')

# Example structure of `imagej_bboxes_df`:
# Columns: ['file name', 'x1', 'y1', 'x2', 'y2']
def calculate_bbox_distance(detection_bbox, imagej_bbox):
    """
    Calculate the Euclidean distance between the centers of two bounding boxes.

    Parameters:
    detection_bbox (tuple): (x1, y1, x2, y2) coordinates of the detection bounding box.
    imagej_bbox (tuple): (x1, y1, x2, y2) coordinates of the ImageJ bounding box.

    Returns:
    float: Euclidean distance between the centers of the bounding boxes.
    """
    # Calculate centers
    detection_center = ((detection_bbox[0] + detection_bbox[2]) / 2, (detection_bbox[1] + detection_bbox[3]) / 2)
    imagej_center = ((imagej_bbox[0] + imagej_bbox[2]) / 2, (imagej_bbox[1] + imagej_bbox[3]) / 2)
    
    # Calculate Euclidean distance between centers
    distance = np.sqrt((detection_center[0] - imagej_center[0])**2 + (detection_center[1] - imagej_center[1])**2)
    return distance
for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    
    # Assume `detections` is a list of bounding boxes from the model
    detection_bboxes = [...]  # Replace with actual detection bounding boxes
    
    # Match with ImageJ bounding box
    imagej_bbox = imagej_bboxes_df[imagej_bboxes_df['file name'] == filename].iloc[0]
    imagej_bbox = (imagej_bbox['x1'], imagej_bbox['y1'], imagej_bbox['x2'], imagej_bbox['y2'])
    
    distances = []
    for detection_bbox in detection_bboxes:
        distance = calculate_bbox_distance(detection_bbox, imagej_bbox)
        distances.append(distance)
    
    # Create a FiftyOne sample and add metadata
    sample = fo.Sample(filepath=img_path)
    sample['bbox_distances'] = distances  # Store all distances for this image
    
    dataset.add_sample(sample)

# Launch FiftyOne app to visualize
session = fo.launch_app(dataset)


# 4. calculate length based on pinhole model

In [ ]:
import numpy as np

def calculate_real_width(focal_length, distance_to_object, width_in_pixels, pixel_size):
    """
    Calculate the real-life width of an object.

    Parameters:
    focal_length (float): Focal length of the camera lens in millimeters (mm).
    distance_to_object (float): Distance from the camera to the object in millimeters (mm).
    width_in_pixels (int): Width of the object in pixels on the image sensor.
    pixel_size (float): Size of a pixel on the image sensor in millimeters (mm).

    Returns:
    float: Real-life width of the object in centimeters (cm).
    """
    # Calculate the width of the object in the image sensor plane in millimeters
    width_in_sensor = width_in_pixels * pixel_size

    # Calculate the real-life width of the object using the similar triangles principle
    real_width_mm = (width_in_sensor * distance_to_object) / focal_length

    # Convert the width from millimeters to centimeters
    real_width_cm = real_width_mm / 10.0

    return real_width_cm

def calculate_euclidean_distance(keypoint1, keypoint2):
    """
    Calculate the Euclidean distance between two keypoints.

    Parameters:
    keypoint1, keypoint2: Tuples of (x, y) coordinates of the keypoints in pixels.

    Returns:
    float: Euclidean distance between the keypoints in pixels.
    """
    return np.sqrt((keypoint1[0] - keypoint2[0])**2 + (keypoint1[1] - keypoint2[1])**2)

# Example of how to integrate this with your FiftyOne dataset
for img_path in image_paths:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    metadata_row = metadata_df[metadata_df['file name'] == filename]
    
    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict()
        sample = fo.Sample(filepath=img_path)
        
        # Add metadata
        for key, value in metadata.items():
            if key != 'file name':
                sample[key] = value
        
        # Calculate the real height or width if keypoints and necessary metadata are available
        if 'height(mm)' in metadata:
            height_mm = metadata['height(mm)']
            focal_length = 6.82  # Example focal length in mm
            pixel_size = 0.0014  # Example pixel size in mm
            keypoints = [...]  # Replace with actual keypoints from your data
            
            if len(keypoints) >= 2:
                # Calculate the Euclidean distance in pixels
                euclidean_distance_pixels = calculate_euclidean_distance(keypoints[0], keypoints[1])
                
                # Calculate the real width/height in centimeters
                real_width_cm = calculate_real_width(focal_length, height_mm, euclidean_distance_pixels, pixel_size)
                
                # Attach the calculated real width/height to the sample
                sample["real_width_cm"] = real_width_cm
        
        dataset.add_sample(sample)

session = fo.launch_app(dataset)


# complete maybe

In [53]:
datasets = fo.list_datasets()

for dataset_name in datasets:
    if dataset_name.startswith('prawn_combined_dataset'):
        fo.delete_dataset(dataset_name)


Could not connect session, trying again in 10 seconds



In [ ]:
import fiftyone as fo
import pandas as pd
import numpy as np
import os
import ast 
from tqdm import tqdm
import json


# Load the existing filtered data with PrawnIDs
filtered_data_file_path = r'C:\Users\gbo10\Videos\research\counting_research_algorithms\src\measurement\ImageJ\Filtered_Data.csv'  # Replace with your actual file path
filtered_df = pd.read_csv(filtered_data_file_path)

# Load the additional metadata from another Excel file
metadata_file_path = r"C:\Users\gbo10\OneDrive\research\thesis and paper\test images.xlsx"  # Replace with your actual file path
metadata_df = pd.read_excel(metadata_file_path)

# Function to parse pose estimation data from a TXT file
def parse_pose_estimation(txt_file):
    pose_estimations = []
    with open(txt_file, 'r') as f:
        for line in f:
            pose_estimations.append([float(x) for x in line.strip().split()])
    return pose_estimations

# Function to calculate Euclidean distance between keypoints
def calculate_euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to calculate the real-life width/height
def calculate_real_width(focal_length, distance_to_object, width_in_pixels, pixel_size):
    width_in_sensor = width_in_pixels * pixel_size
    real_width_mm = (width_in_sensor * distance_to_object) / focal_length
    return real_width_mm
def extract_identifier_from_gt(filename):
    return filename.split('-')[0]

folder_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\car'  # Replace with the folder containing the images and TXT files

# Assuming you have the following lists
image_paths = [os.path.join(folder_path, image) for image in os.listdir(folder_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

prediction_folder_path=r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict19\labels"

txt_paths = [os.path.join(prediction_folder_path, txt) for txt in os.listdir(prediction_folder_path) if txt.endswith('.txt')]
# Create a FiftyOne dataset
ground_truth_paths = r"C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\labels"  # Replace with the folder containing the ground truth TXT files

ground_truth_paths_text = [os.path.join(ground_truth_paths, txt) for txt in os.listdir(ground_truth_paths) if txt.endswith('.txt')]


import fiftyone as fo

# List all datasets in FiftyOne

# Loop through the list and delete datasets that match the condition


#delete all datasets


# Create a new dataset

dataset = fo.Dataset("prawn_combined_dataset22", overwrite=True)

dataset.default_skeleton = fo.KeypointSkeleton(
    labels=[
        "start_carapace",
        "eyes",
    ],
    edges=[
        [0, 1],  # Connect keypoint 0 to keypoint 1  # Connect keypoint 1 to keypoint 2
        # Add more connections as needed
    ],
)
# Loop over images and corresponding TXT files
for img_path in image_paths:

    # Extract the relevant part for matching
    filename = os.path.splitext(os.path.basename(img_path))[0] 
    parts = filename.split('_')
    relevant_part = f"{parts[1][-3:]}_{parts[3].split('.')[0]}"
     # e.g., undistorted_GX010152_36_378.jpg_gamma
    identifier = filename.replace('undistorted_', '').replace('.jpg_gamma', '')  # Extract the identifier from the filename


    # Construct the paths to the prediction and ground truth files
    prediction_txt_path = os.path.join(prediction_folder_path, f"{filename}.txt")

    # Match ground truth based on the extracted identifier
    ground_truth_txt_path = None
    for gt_file in ground_truth_paths_text:
        b= extract_identifier_from_gt(os.path.basename(gt_file))
        if b == identifier:
            ground_truth_txt_path = gt_file

            break

    
    # Parse the pose estimation data from the TXT file
    pose_estimations = parse_pose_estimation(prediction_txt_path)
    
    ground_truths = parse_pose_estimation(ground_truth_txt_path)

    
    keypoints_list = []  # List to store keypoints
    detections = []  # List to store detection bounding boxes
    ground_truth_keypoints_list = []
    ground_truth_detection_list = []

    for pose in pose_estimations:
        if len(pose)==11:        

            x1_rel = pose[1] 
            y1_rel = pose[2] 
            width_rel = pose[3] 
            height_rel = pose[4] 

            
            #center to top left
            x1_rel = x1_rel - width_rel/2
            y1_rel = y1_rel - height_rel/2
        # Calc



            keypoints = []
            confidences = []
            for i in range(5, len(pose), 3):
                x_kp_scaled = pose[i] 
                y_kp_scaled = pose[i + 1] 
                confidence = pose[i + 2]
                keypoints.append([x_kp_scaled, y_kp_scaled])
            

            confidences = [float(pose[i+2]) for i in range(5, len(pose), 3)]


         
                
            keypoint=fo.Keypoint(points=keypoints, confidence=None)
            keypoints_list.append(keypoint)


            keypoints_dict= {'point1':keypoints[0],'point2':keypoints[1]}
            detections.append(fo.Detection
                              (label="prawn", bounding_box=[x1_rel, y1_rel, width_rel, height_rel],
                               attributes={'keypoints':keypoints_dict}))
            
    for gt_pose in ground_truths:
            x1_rel = gt_pose[1]
            y1_rel = gt_pose[2]
            width_rel = gt_pose[3]
            height_rel = gt_pose[4]

            # Convert center to top-left
            x1_rel = x1_rel - width_rel / 2
            y1_rel = y1_rel - height_rel / 2

            keypoints = []
            for i in range(5, len(gt_pose), 3):
                x_kp_scaled = gt_pose[i]
                y_kp_scaled = gt_pose[i + 1]
                keypoints.append([x_kp_scaled, y_kp_scaled])

            ground_truth_keypoints_list.append(fo.Keypoint(points=keypoints))
            ground_truth_detection_list.append(fo.Detection(label="prawn_truth", bounding_box=[x1_rel, y1_rel, width_rel, height_rel]))

    sample = fo.Sample(filepath=img_path)
    
    sample["ground_truth_keypoints"] = fo.Keypoints(keypoints=ground_truth_keypoints_list)


    sample["ground_truth_detections"] = fo.Detections(detections=detections)   
    # Add keypoints as a separate field
    sample["keypoints"] = fo.Keypoints(keypoints=keypoints_list)
   
    sample["detections_predictions"] = fo.Detections(detections=detections)
    


    
    
    # Match the filename with the corresponding rows in the filtered data
    matching_rows = filtered_df[filtered_df['Label'] == f'carapace:{filename}']
    
    # Add metadata from the additional Excel file
    metadata_row = metadata_df[metadata_df['file name'] == relevant_part]

    if not metadata_row.empty:
        metadata = metadata_row.iloc[0].to_dict() 
        for key, value in metadata.items():
            if key != 'file name': 
                sample[key] = value
            
                        
    else:
        print(f"No metadata found for {relevant_part}")
    true_detections=[]

    for _, row in matching_rows.iterrows():
        prawn_id = row['PrawnID']
        prawn_bbox = ast.literal_eval(row['BoundingBox_1'])  # Replace with the correct column name

        # Convert the tuple elements to floats
        prawn_bbox = tuple(float(coord) for coord in prawn_bbox)# Replace with correct bounding box columns
        
        prawn_normalized_bbox=[prawn_bbox[0]/5312, prawn_bbox[1]/2988, prawn_bbox[2]/5312, prawn_bbox[3]/2988]

        # Convert PrawnID bounding box to absolute coordinates
        
        
        true_detections.append(fo.Detection(label="prawn_true", bounding_box=prawn_normalized_bbox))

        prawn_point=(prawn_bbox[0]/5312, prawn_bbox[1]/2988)
        

        # Find the closest detection bounding box within the same image (Label)
        min_distance = float('inf')
        closest_detection = None
        
        for detection_bbox in sample["detections_predictions"].detections:
            detection_bbox_coords = detection_bbox.bounding_box
            
            det_point=(detection_bbox_coords[0], detection_bbox_coords[1])
            
            distance = calculate_euclidean_distance(prawn_point, det_point)
            
            if distance < min_distance:
                min_distance = distance
                closest_detection = detection_bbox
        
        if closest_detection is not None:
            height_mm = sample['heigtht(mm)']  # Use appropriate column for height in mm
            focal_length = 24.22
            pixel_size = 0.00716844 

                

            keypoints_dict2 = closest_detection.attributes["keypoints"]

            keypoints1 = [keypoints_dict2['point1'], keypoints_dict2['point2']]    

            keypoint1_scaled = [keypoints1[0][0] * 5312, keypoints1[0][1] * 2988]
            keypoint2_scaled = [keypoints1[1][0] * 5312, keypoints1[1][1] * 2988]

            euclidean_distance_pixels = calculate_euclidean_distance(keypoint1_scaled, keypoint2_scaled)
            real_length_cm = calculate_real_width(focal_length, height_mm, euclidean_distance_pixels, pixel_size)
            

            # Update the filtered_df with the calculated lengths
            filtered_df.loc[(filtered_df['Label'] == f'carapace:{filename}') & (filtered_df['PrawnID'] == prawn_id), 'RealLength(cm)'] = real_length_cm
            
            # Add floating text near the keypoints in FiftyOne

            true_length=filtered_df.loc[(filtered_df['Label'] == f'carapace:{filename}') & (filtered_df['PrawnID'] == prawn_id), 'Avg_Length'].values[0]


            closest_detection_label =f'MPError: {abs(real_length_cm - true_length) / true_length * 100:.2f}%, true length: {true_length:.2f}cm, pred length: {real_length_cm:.2f}cm'
            
            closest_detection.label=closest_detection_label


            #if mpe>25: tag sample
            if abs(real_length_cm - true_length) / true_length * 100 > 25:
                if "MPE>25" not in sample.tags:
                    sample.tags.append("MPE>25")
            
    # Assuming 'sample' is your FiftyOne sample object
        
    sample["true_detections"] = fo.Detections(detections=true_detections)            

    dataset.add_sample(sample)

ground_truth_count=0

for sample in dataset:
    ground_truth_count+=len(sample["ground_truth_detections"].detections)

print('gt',ground_truth_count)

detections_predictions_count=0

for sample in dataset:
    detections_predictions_count+=len(sample["detections_predictions"].detections)

print('pred',detections_predictions_count)


# Save the updated filtered data with the calculated lengths
# filtered_df.to_excel('updated_filtered_data_with_lengths.xlsx', index=False)

# Launch the FiftyOne app to visualize

# Define a custom view

# Configure the view to show your custom attributes


session = fo.launch_app(dataset, port=5151)


# Launch the app with the custom view



In [27]:
import numpy as np
import cv2
from ultralytics import SAM
from math import sqrt
from random import randint, shuffle
import fiftyone as fo
import fiftyone.core.labels as fol
import os

# Helper Classes and Functions for Circle Calculation
class Point:
    def __init__(self, X=0, Y=0) -> None:
        self.X = X
        self.Y = Y

class Circle:
    def __init__(self, c=Point(), r=0) -> None:    
        self.C = c
        self.R = r

def dist(a, b):
    return sqrt((a.X - b.X) ** 2 + (a.Y - b.Y) ** 2)

def is_inside(c, p):
    return dist(c.C, p) <= c.R

def get_circle_center(bx, by, cx, cy):
    B = bx * bx + by * by
    C = cx * cx + cy * cy
    D = bx * cy - by * cx
    return Point((cy * B - by * C) / (2 * D), (bx * C - cx * B) / (2 * D))

def circle_from1(A, B):
    C = Point((A.X + B.X) / 2.0, (A.Y + B.Y) / 2.0)
    return Circle(C, dist(A, B) / 2.0)

def circle_from2(A, B, C):
    I = get_circle_center(B.X - A.X, B.Y - A.Y, C.X - A.X, C.Y - A.Y)
    I.X += A.X
    I.Y += A.Y
    return Circle(I, dist(I, A))

def is_valid_circle(c, P):
    for p in P:
        if not is_inside(c, p):
            return False
    return True

def min_circle_trivial(P):
    assert len(P) <= 3
    if not P:
        return Circle()
    elif len(P) == 1:
        return Circle(P[0], 0)
    elif len(P) == 2:
        return circle_from1(P[0], P[1])
    for i in range(3):
        for j in range(i + 1, 3):
            c = circle_from1(P[i], P[j])
            if is_valid_circle(c, P):
                return c
    return circle_from2(P[0], P[1], P[2])

def welzl_helper(P, R, n):
    if n == 0 or len(R) == 3:
        return min_circle_trivial(R)
    idx = randint(0, n - 1)
    p = P[idx]
    P[idx], P[n - 1] = P[n - 1], P[idx]
    d = welzl_helper(P, R.copy(), n - 1)
    if is_inside(d, p):
        return d
    R.append(p)
    return welzl_helper(P, R.copy(), n - 1)

def welzl(P):
    P_copy = P.copy()
    shuffle(P_copy)
    return welzl_helper(P_copy, [], len(P_copy))
def yolo_key_to_box(yolo_line, img_width=640, img_height=640):
    # YOLO format: class x_center y_center width height (normalized)
    yolo_data = yolo_line.strip().split()
    x_center, y_center, width, height = map(float, yolo_data[1:5])


    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height

    # Calculate top-left and bottom-right coordinates
    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return np.array([ x_min, y_min, x_max, y_max])

def convert_yolo_key_to_boxes(yolo_file):
    boxes = []
    with open(yolo_file, 'r') as file:
        for line in file:
            box = yolo_key_to_box(line)
            boxes.append(box)
    return boxes

def find_matching_yolo_file(core_name, yolo_dir):
    for yolo_file in os.listdir(yolo_dir):
        if core_name in yolo_file:  # Check if the core name is part of the YOLO file name
            return os.path.join(yolo_dir, yolo_file)
    return None 




# Initialize FiftyOne dataset
dataset = fo.Dataset("prawn_segmentation", overwrite=True)

# Load SAM model
model = SAM(r'mobile_sam.pt')

# Image directory and YOLO annotation files
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images"
yolo_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict19\labels"

# Iterate over images
for image_name in os.listdir(image_dir):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_dir, image_name)
       
        core_name = image_name.split('.')[0]
        # Create the corresponding YOLO file path

                

        yolo_file = find_matching_yolo_file(core_name, yolo_dir)
        
        # Convert YOLO annotations to bounding boxes
        boxes = convert_yolo_key_to_boxes(yolo_file)
        
        # Load image
        image = cv2.imread(image_path)
        
        # Perform segmentation
        
        # Initialize list to store segmentations
        all_polygons = []
        all_attributes = []

        
        # Process each mask individually
        for xy in results[0].masks.xy:
                 
    # Normalize points to [0, 1] range
                normalized_points = [(float(p[0]) / image.shape[1], float(p[1]) / image.shape[0]) for p in xy]
                all_polygons.append(normalized_points)
                points = [Point(p[0], p[1]) for p in xy]
                mec = welzl(points)
                diameter = mec.R * 2 
                all_attributes.append({'diameter': diameter})
                # Convert contour to a format suitable for FiftyOne
                segmentation = fol.Polyline(
                        points=all_polygons,
                        closed=True,
                        filled=True,
                        attributes=all_attributes

                        
                    )
        
        # Add image and segmentation to FiftyOne dataset
        sample = fo.Sample(filepath=image_path)
        sample["segmentations"] =fol.Polylines(polylines=[segmentation])

        dataset.add_sample(sample)

# Save dataset
dataset.save()
session = fo.launch_app(dataset, port=5153)


0: 1024x1024 1617.6ms
Speed: 23.0ms preprocess, 1617.6ms inference, 6.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1602.6ms
Speed: 18.6ms preprocess, 1602.6ms inference, 8.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1428.8ms
Speed: 19.0ms preprocess, 1428.8ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1512.6ms
Speed: 22.0ms preprocess, 1512.6ms inference, 4.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1498.9ms
Speed: 23.9ms preprocess, 1498.9ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1508.2ms
Speed: 22.5ms preprocess, 1508.2ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1320.1ms
Speed: 19.0ms preprocess, 1320.1ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1388.6ms
Speed: 20.0ms preprocess, 1388.6ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1024)


RuntimeError: Client is not connected

In [2]:
import os
import cv2
import numpy as np
from ultralytics import SAM

# YOLO format conversion
def yolo_key_to_box(yolo_line, img_width=640, img_height=640):
    yolo_data = yolo_line.strip().split()
    x_center, y_center, width, height = map(float, yolo_data[1:5])

    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height

    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    return np.array([x_min, y_min, x_max, y_max])

def convert_yolo_key_to_boxes(yolo_file):
    boxes = []
    with open(yolo_file, 'r') as file:
        for line in file:
            box = yolo_key_to_box(line)
            boxes.append(box)
    return boxes

def find_matching_yolo_file(core_name, yolo_dir):
    for yolo_file in os.listdir(yolo_dir):
        if core_name in yolo_file:  # Check if the core name is part of the YOLO file name
            return os.path.join(yolo_dir, yolo_file)
    return None 

# Initialize SAM model
model = SAM(r'C:\Users\gbo10\Videos\research\counting_research_algorithms\fifty_one\measurements\sam2_b.pt')

# Image directory and YOLO annotation files
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted_resized"
yolo_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict39\labels"
# Iterate over images
for image_name in os.listdir(image_dir):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_dir, image_name)

        core_name = image_name.split('.')[0]
        
        # Create the corresponding YOLO file path
        yolo_file = find_matching_yolo_file(core_name, yolo_dir)
        if yolo_file is None:
            print(f"No matching YOLO file found for {image_name}")
            continue
        
        # Convert YOLO annotations to bounding boxes
        boxes = convert_yolo_key_to_boxes(yolo_file)
        
        # Load image
        image = cv2.imread(image_path)
        
        # Perform segmentation
        results = model(image, imgsz=640,save=True, bboxes=np.array(boxes))
        
        # Prepare a .txt file to save the segmentations
        output_txt_path = os.path.join(image_dir, f"{core_name}_segmentations.txt")
        
        with open(output_txt_path, 'w') as f:
            # Process each mask individually
            for xy in results[0].masks.xy:
                # Convert the xy coordinates into a string format for saving
                xy_str = " ".join(map(str, xy.flatten().tolist()))  # Flatten and convert to string
                f.write(xy_str + "\n")  # Write each xy as a line
                print(f"Saved segmentation for {image_name} to {output_txt_path}")



0: 1024x1024 5168.0ms
Speed: 54.0ms preprocess, 5168.0ms inference, 1.0ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\segment\predict44
Saved segmentation for undistorted_vlcsnap-2024-09-19-21h46m36s160.png to C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted_resized\undistorted_vlcsnap-2024-09-19-21h46m36s160_segmentations.txt

0: 1024x1024 8094.8ms
Speed: 17.0ms preprocess, 8094.8ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to c:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\segment\predict44
Saved segmentation for undistorted_vlcsnap-2024-09-19-21h46m54s892.png to C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted_resized\undistorted_vlcsnap-2024-09-19-21h46m54s892_segmentations.txt
Saved segmentation for undistorted_vlcsnap-2024-09-19-21h46m54s892.png to C:\Users\gbo10\OneDrive\measur

In [6]:
import os
import cv2
import numpy as np

def draw_filled_segmentations_with_opacity(image, xy_coordinates, color=(0, 255, 0), opacity=0.2):
    """
    Draws and fills the segmentation polygons on the image with a specified opacity.
    xy_coordinates: List of lists where each inner list represents a segmentation polygon 
                    (each polygon is a list of [x, y] pairs).
    color: The color to fill the polygons (in BGR format).
    opacity: A float value between 0 and 1 that determines the transparency of the fill.
    """
    # Create an overlay image to draw on
    overlay = image.copy()
    
    # Draw filled polygons on the overlay
    for polygon in xy_coordinates:
        polygon = np.array(polygon, np.int32)  # Convert to NumPy array of int32
        polygon = polygon.reshape((-1, 1, 2))  # Reshape for OpenCV
        cv2.fillPoly(overlay, [polygon], color=color)  # Fill the polygon on the overlay
    
    # Blend the overlay with the original image
    cv2.addWeighted(overlay, opacity, image, 1 - opacity, 0, image)
    return image

def load_segmentations_from_txt(txt_file):
    """
    Loads segmentation polygons from a .txt file.
    Each line in the file contains space-separated coordinates: x1 y1 x2 y2 ... xn yn.
    """
    segmentations = []
    with open(txt_file, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))  # Read all floats from the line
            polygon = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]  # Convert into (x, y) pairs
            segmentations.append(polygon)
    return segmentations

# Specify the image and corresponding segmentation file
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left_resized"
image_name = "undistorted_GX010091_5_149.jpg_gamma.jpg"  # Replace with the name of your image
image_path = os.path.join(image_dir, image_name)

core_name = image_name.split('.')[0]

# Path to the corresponding segmentations .txt file
txt_file_path = os.path.join(image_dir, f"{core_name}_segmentations.txt")

# Load the image
image = cv2.imread(image_path)

# Check if the segmentation file exists
if os.path.exists(txt_file_path):
    # Load segmentations from the .txt file
    segmentations = load_segmentations_from_txt(txt_file_path)
    
    # Draw and fill the segmentations on the image
    image_with_segmentations = draw_filled_segmentations_with_opacity(image, segmentations)
    
    # Save or display the result
    output_image_path = os.path.join(image_dir, f"{core_name}_with_segmentations.png")
    cv2.imwrite(output_image_path, image_with_segmentations)
    print(f"Saved image with filled segmentations to {output_image_path}")
    
    # Optionally, display the image (for local testing)
    # cv2.imshow('Segmented Image', image_with_segmentations)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
else:
    print(f"No segmentation file found for {image_name}")


Saved image with filled segmentations to C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left_resized\undistorted_GX010091_5_149_with_segmentations.png


# try skeletonization

In [24]:
import cv2
import numpy as np
from skimage.morphology import skeletonize
import networkx as nx

def load_multiple_coords_from_txt(txt_file):
    """
    Loads multiple (x, y) coordinates sets from a .txt file.
    Each line in the file contains space-separated coordinates: x1 y1 x2 y2 ... xn yn.
    Each line represents a different segmentation.
    """
    segmentations = []
    with open(txt_file, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))  # Read all floats from the line
            polygon = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]   # Convert into (y, x) pairs
            segmentations.append(polygon)
    return segmentations

def create_filled_binary_mask(coords, img_height, img_width):
    """
    Creates a binary mask with a filled polygon from a single segmentation's (y, x) coordinates.
    """
    binary_mask = np.zeros((img_height, img_width), dtype=np.uint8)
    polygon = np.array(coords, np.int32)  # Convert to NumPy array of int32
    polygon = polygon.reshape((-1, 1, 2))  # Reshape for OpenCV fillPoly
    cv2.fillPoly(binary_mask, [polygon], color=1) 
    
     # Fill the polygon on the binary mask
    return binary_mask

def skeletonize_mask(binary_mask,i):
    """
    Skeletonizes the binary mask using skimage's skeletonize function.
    """
    skeleton = skeletonize(binary_mask)
    cv2.imwrite(f'skeleton{i}.png', skeleton.astype(np.uint8) * 255)

    return skeleton

    

def draw_skeleton(image, skeleton_coords, color=(0, 255, 0), thickness=1):
    """
    Draws the skeleton on the image.
    skeleton_coords: List of (y, x) points representing the skeleton.
    color: The color to draw the skeleton (in BGR format).
    thickness: The thickness of the lines representing the skeleton.
    """
    for i in range(len(skeleton_coords) - 1):
        start_point = (skeleton_coords[i][1], skeleton_coords[i][0])  # (x, y)
        end_point = (skeleton_coords[i+1][1], skeleton_coords[i+1][0])  # (x, y)
        cv2.line(image, start_point, end_point, color=color, thickness=thickness)
    return image

def draw_longest_path(image, longest_path, color=(0, 0, 255), thickness=2):
    """
    Draws the longest path on the image in a different color.
    longest_path: List of (y, x) points representing the longest path.
    color: The color to draw the longest path (in BGR format).
    thickness: The thickness of the lines representing the longest path.
    """
    for i in range(len(longest_path) - 1):
        start_point = (longest_path[i][1], longest_path[i][0])  # (x, y)
        end_point = (longest_path[i+1][1], longest_path[i+1][0])  # (x, y)
        cv2.line(image, start_point, end_point, color=color, thickness=thickness)
    return image

def find_longest_path(skeleton_coords):
    """
    Finds the longest path in the skeleton using graph analysis.
    skeleton_coords: List of (y, x) points representing the skeleton.
    """
    G = nx.Graph()
    for y, x in skeleton_coords:
        G.add_node((y, x))  # Add node at each skeleton point
        for dy in [-1, 0, 1]:
            for dx in [-1, 0, 1]:
                if dy == 0 and dx == 0:
                    continue  # Skip the current point itself
                neighbor = (y + dy, x + dx)
                if neighbor in G.nodes:
                    G.add_edge((y, x), neighbor)

    connected_components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    largest_component = max(connected_components, key=len)
    endpoints = [node for node, degree in dict(largest_component.degree()).items() if degree == 1]

    max_length = 0
    longest_path = []

    for source in endpoints:
        lengths = nx.single_source_shortest_path_length(largest_component, source)
        farthest_node = max(lengths, key=lengths.get)
        length = lengths[farthest_node]
        if length > max_length:
            max_length = length
            longest_path = nx.shortest_path(largest_component, source, farthest_node)

    return longest_path

image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\right_resized"
image_name = "undistorted_GX010067_33_625.jpg_gamma.jpg"  # Replace with the name of your image
image_path = os.path.join(image_dir, image_name)

core_name = image_name.split('.')[0]

# Path to the corresponding segmentations .txt file
txt_file_path = os.path.join(image_dir, f"{core_name}_segmentations.txt")  # Replace with your .txt file path

# Load image and segmentations
image = cv2.imread(image_path)
segmentations = load_multiple_coords_from_txt(txt_file_path)
img_height, img_width = image.shape[:2]

i=0
# Process each segmentation individually
for coords in segmentations:
   
    binary_mask = create_filled_binary_mask(coords, img_height, img_width)
    skeleton = skeletonize_mask(binary_mask,i)
    skeleton_coords = np.column_stack(np.nonzero(skeleton))
    longest_path = find_longest_path(skeleton_coords)
    
    # Draw skeleton and longest path
    image = draw_skeleton(image, skeleton_coords)
    image = draw_longest_path(image, longest_path)
    
# Save or display the final output image as needed
cv2.imwrite('output_image_with_skeleton_and_longest_path.png', image)


True

In [ ]:

image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\right_resized"
image_name = "undistorted_GX010067_33_625.jpg_gamma.jpg"  # Replace with the name of your image
image_path = os.path.join(image_dir, image_name)

core_name = image_name.split('.')[0]

# Path to the corresponding segmentations .txt file
txt_file_path = os.path.join(image_dir, f"{core_name}_segmentations.txt") 

# end here

In [9]:
import os

# Image directory and YOLO annotation files
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images"
yolo_dir = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict19\labels"

# Helper function to extract the core image name
def extract_core_image_name(image_name):
    # Remove everything after the first hyphen
    core_name = image_name.split('-')[0]
    
    # Split the core name by underscores, and capture the first three parts
    parts = core_name.split('_')
    
    # Rebuild the core name using the first three parts
    core_name = '_'.join(parts[:3])
    
    return core_name

# Helper function to find the matching YOLO file
def find_matching_yolo_file(core_name, yolo_dir):
    for yolo_file in os.listdir(yolo_dir):
        if core_name in yolo_file:  # Check if the core name is part of the YOLO file name
            return os.path.join(yolo_dir, yolo_file)
    return None  # Return None if no match is found

# Helper function to rename images
def rename_images(image_dir, yolo_dir):
    for image_name in os.listdir(image_dir):
        if image_name.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_dir, image_name)
            
            # Extract the core name from the image filename
            core_name = extract_core_image_name(image_name)
            
            # Find the matching YOLO file
            yolo_file_path = find_matching_yolo_file(core_name, yolo_dir)
            
            if yolo_file_path:
                # Create the new image name using the core name
                new_image_name = f"undistorted_{core_name}.jpg"
                new_image_path = os.path.join(image_dir, new_image_name)
                
                # Rename the image
                print(f"Renaming {image_path} to {new_image_path}")
                os.rename(image_path, new_image_path)
            else:
                print(f"No matching YOLO file for {image_name}, skipping...")

# Run the renaming process
rename_images(image_dir, yolo_dir)


Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010152_36_378-jpg_gamma_jpg.rf.dde16353cb779b5cc6dabcd884e15547.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\undistorted_GX010152_36_378.jpg
Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010155_18_219-jpg_gamma_jpg.rf.9057ffadeef9e786b3936806a3c9f36a.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\undistorted_GX010155_18_219.jpg
Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010155_78_1170-jpg_gamma_jpg.rf.e004afd947d9bd621eb3843251abf362.jpg to C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\undistorted_GX010155_78_1170.jpg
Renaming C:\Users\gbo10\OneDrive\measurement_paper_images\car to compare\tester_choise\test\images\GX010157_174_2582

 # current carapace

In [2]:
# main.py
import fiftyone as fo
import os
from data_loader import load_data, create_dataset, process_images
from utils import parse_pose_estimation, calculate_euclidean_distance, calculate_real_width, extract_identifier_from_gt
# from measurements.metrics import  compute_dataset_map


import sys
from importlib import reload


# Reload the module
reload(sys.modules['data_loader'])
reload(sys.modules['utils'])


# Paths to data files
filtered_data_file_path = r'C:\Users\gbo10\Videos\research\counting_research_algorithms\src\measurement\ImageJ\Filtered_Data.csv'
metadata_file_path = r"C:\Users\gbo10\OneDrive\research\thesis and paper\test images.xlsx"
right_folder_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\right'
right_prediction_folder_path = r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict35\labels"
right_ground_truth_folder_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\current from roboflow -28.8-first sense\measurement\test\test-right\valid\labels"

left_folder_path=r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left'
left_prediction_folder_path=r"C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict36\labels"
left_ground_truth_folder_path=r"C:\Users\gbo10\OneDrive\measurement_paper_images\current from roboflow -28.8-first sense\measurement\test\test-left\valid\labels"

car_folder_path=r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\car'
car_prediction_folder_path=r'C:\Users\gbo10\Videos\data-science\Research-counting-algorithms\runs\pose\predict38\labels'
car_ground_truth_folder_path=r'C:\Users\gbo10\OneDrive\measurement_paper_images\current from roboflow -28.8-first sense\measurement\test\test-square\valid\labels'


# Load data and metadata
filtered_df, metadata_df = load_data(filtered_data_file_path, metadata_file_path)

# Create FiftyOne dataset
dataset = create_dataset()

for folder_path, prediction_folder_path, ground_truth_folder_path in zip([right_folder_path, left_folder_path, car_folder_path],
                                                                         [right_prediction_folder_path, left_prediction_folder_path, car_prediction_folder_path],
                                                                         [right_ground_truth_folder_path, left_ground_truth_folder_path, car_ground_truth_folder_path]):
    # Get image paths
    image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.endswith(('.png', '.jpg', '.jpeg'))]

    # Get ground truth paths
    ground_truth_paths = [os.path.join(ground_truth_folder_path, txt) for txt in os.listdir(ground_truth_folder_path) if txt.endswith('.txt')]

    prediction_paths=[os.path.join(prediction_folder_path, txt) for txt in os.listdir(prediction_folder_path) if txt.endswith('.txt')]

    
    # Process images and predictions using segmentations and bounding boxes
    pond_tag=folder_path.split('/')[-1]
    print(pond_tag)
    process_images(image_paths=image_paths,ground_truth_paths_text=ground_truth_paths,prediction_folder_path= prediction_folder_path,filtered_df= filtered_df,metadata_df= metadata_df,dataset= dataset,pond_type=pond_tag)

# Process images and predictions




# Start the FiftyOne session
session = fo.launch_app(dataset, port=5154)


1\carapace\right


100%|██████████| 37/37 [00:01<00:00, 20.25it/s]


1\carapace\left


100%|██████████| 5/5 [00:00<00:00, 25.52it/s]


1\carapace\car


...

 33%|███▎      | 10/30 [00:00<00:00, 39.40it/s]

No ground truth found for undistorted_GX010169_121_1249
No ground truth found for undistorted_GX010174_62_790.jpg_gamma
No ground truth found for undistorted_GX010175_266_3372.jpg_gamma.jpg_gamma
No ground truth found for undistorted_GX010175_82_852.jpg_gamma
No ground truth found for undistorted_GX010177_232_3047


 63%|██████▎   | 19/30 [00:00<00:00, 52.57it/s]

No ground truth found for undistorted_GX010178_114_2258
No ground truth found for undistorted_GX010178_172_3604.jpg_gamma
No ground truth found for undistorted_GX010179_200_3927.jpg_gamma


100%|██████████| 30/30 [00:00<00:00, 40.65it/s]


['c:\\Users\\gbo10\\Videos\\research\\counting_research_algorithms\\fifty_one', 'C:\\Users\\gbo10\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip', 'C:\\Users\\gbo10\\AppData\\Local\\Programs\\Python\\Python39\\DLLs', 'C:\\Users\\gbo10\\AppData\\Local\\Programs\\Python\\Python39\\lib', 'C:\\Users\\gbo10\\AppData\\Local\\Programs\\Python\\Python39', 'c:\\Users\\gbo10\\Videos\\research\\counting_research_algorithms\\.venv', '', 'c:\\Users\\gbo10\\Videos\\research\\counting_research_algorithms\\.venv\\lib\\site-packages', 'c:\\Users\\gbo10\\Videos\\research\\counting_research_algorithms\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\gbo10\\Videos\\research\\counting_research_algorithms\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\gbo10\\Videos\\research\\counting_research_algorithms\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\gbo10\\Videos\\research\\counting_research_algorithms\\fifty_one']


# add swimming


In [7]:
import numpy as np
import ast
import pandas as pd


# Helper function to calculate Euclidean distance between two bounding box points
def calculate_euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to check if the bounding boxes are close enough based on a distance threshold
def find_closest_swimming_bbox(prawn_bbox, swimming_rows, threshold=50):
    prawn_point = (prawn_bbox[0], prawn_bbox[1])  # Top-left corner of the prawn bounding box
    min_distance = float('inf')
    closest_swimming_bbox = None

    # Iterate through all swimming bounding boxes
    for _, swim_row in swimming_rows.iterrows():
        swim_point = (swim_row['BX'], swim_row['BY'])  # Top-left corner of the swimming bounding box

        # Calculate the Euclidean distance
        distance = calculate_euclidean_distance(prawn_point, swim_point)

        # If the distance is smaller than the current minimum, update the closest match
        if distance < min_distance:
            min_distance = distance
            closest_swimming_bbox = swim_row

    # Return True if a close enough match is found, otherwise False
    return min_distance < threshold

# Add the swimming status to the updated dataframe
def add_swimming_status(updated_df, swimming_df, threshold=50):
    # Initialize the swimming column with default value "not swimming"
    updated_df['swimming'] = 'not swimming'

    # Iterate over each prawn in updated_df
    for i, row in updated_df.iterrows():
        prawn_label = row['Label'].split(':')[1]  # Extract the part after 'carapace:' from prawn label
        prawn_bbox = ast.literal_eval(row['BoundingBox_1'])  # Convert bounding box string to a tuple

        # Get corresponding rows from swimming_df with the same Label
        matching_swimming_rows = swimming_df[swimming_df['Label'].apply(lambda x: x.split(':')[1]) == prawn_label]

        # Check each matching row in swimming_df to see if the bounding boxes are close enough
        if not matching_swimming_rows.empty:
            if find_closest_swimming_bbox(prawn_bbox, matching_swimming_rows, threshold):
                # If a close match is found, mark the prawn as swimming
                updated_df.at[i, 'swimming'] = 'swimming'

    return updated_df

# Load the updated dataframe and the swimming dataframe
updated_df = pd.read_excel(r'C:\Users\gbo10\Videos\research\counting_research_algorithms\fifty_one\measurements\Updated_Filtered_Data_with_real_length.xlsx')
swimming_df = pd.read_excel(r'C:\Users\gbo10\Videos\research\counting_research_algorithms\src\measurement\ImageJ\Swimming.xlsx')

# Add the swimming status column
updated_df_with_swimming = add_swimming_status(updated_df, swimming_df)

# Save the updated dataframe to a new Excel file
updated_df_with_swimming.to_excel('Updated_Filtered_Data_with_Swimming.xlsx', index=False)


AttributeError: 'Series' object has no attribute 'split'

# body length

In [ ]:
import os
import cv2
import numpy as np
from math import sqrt
from random import randint, shuffle
import fiftyone as fo
import fiftyone.core.labels as fol

# Helper Classes and Functions for Circle Calculation
class Point:
    def __init__(self, X=0, Y=0) -> None:
        self.X = X
        self.Y = Y

class Circle:
    def __init__(self, c=Point(), r=0) -> None:    
        self.C = c
        self.R = r

def dist(a, b):
    return sqrt((a.X - b.X) ** 2 + (a.Y - b.Y) ** 2)

def is_inside(c, p):
    return dist(c.C, p) <= c.R

def get_circle_center(bx, by, cx, cy):
    B = bx * bx + by * by
    C = cx * cx + cy * cy
    D = bx * cy - by * cx
    return Point((cy * B - by * C) / (2 * D), (bx * C - cx * B) / (2 * D))

def circle_from1(A, B):
    C = Point((A.X + B.X) / 2.0, (A.Y + B.Y) / 2.0)
    return Circle(C, dist(A, B) / 2.0)

def circle_from2(A, B, C):
    I = get_circle_center(B.X - A.X, B.Y - A.Y, C.X - A.X, C.Y - A.Y)
    I.X += A.X
    I.Y += A.Y
    return Circle(I, dist(I, A))

def is_valid_circle(c, P):
    for p in P:
        if not is_inside(c, p):
            return False
    return True

def min_circle_trivial(P):
    assert len(P) <= 3
    if not P:
        return Circle()
    elif len(P) == 1:
        return Circle(P[0], 0)
    elif len(P) == 2:
        return circle_from1(P[0], P[1])
    for i in range(3):
        for j in range(i + 1, 3):
            c = circle_from1(P[i], P[j])
            if is_valid_circle(c, P):
                return c
    return circle_from2(P[0], P[1], P[2])

def welzl_helper(P, R, n):
    if n == 0 or len(R) == 3:
        return min_circle_trivial(R)
    idx = randint(0, n - 1)
    p = P[idx]
    P[idx], P[n - 1] = P[n - 1], P[idx]
    d = welzl_helper(P, R.copy(), n - 1)
    if is_inside(d, p):
        return d
    R.append(p)
    return welzl_helper(P, R.copy(), n - 1)

def welzl(P):
    P_copy = P.copy()
    shuffle(P_copy)
    return welzl_helper(P_copy, [], len(P_copy))

# Helper function to load segmentations from a .txt file
def load_segmentations_from_txt(txt_file):
    """
    Loads segmentation polygons from a .txt file without scaling the coordinates.
    Each line in the file contains space-separated coordinates: x1 y1 x2 y2 ... xn yn.
    Assumes coordinates are already scaled to 640x640.
    """
    segmentations = []
    with open(txt_file, 'r') as file:
        for line in file:
            coords = list(map(float, line.strip().split()))  # Read all floats from the line
            polygon = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]  # Directly read the coordinates
            segmentations.append(polygon)
    return segmentations


# Initialize FiftyOne dataset
dataset = fo.Dataset("prawn_segmentation", overwrite=True)

# Image directory and YOLO annotation files
# Assuming image size is always 640x640 based on your .txt files

# Image directory and YOLO annotation files
image_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left_resized"
segmentation_dir = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left_resized" # Directory with the segmentation .txt files

# Iterate over images
for image_name in os.listdir(image_dir):
    if image_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_dir, image_name)
       
        core_name = image_name.split('.')[0]
        # Create the corresponding segmentation txt file path
        segmentation_file = os.path.join(segmentation_dir, f"{core_name}_segmentations.txt")
        
        if not os.path.exists(segmentation_file):
            print(f"Segmentation file not found for {core_name}")
            continue
        
        # Load image (assuming it's also 640x640)
        image = cv2.imread(image_path)
        
        # Load segmentations from the .txt file (no scaling needed)
        all_polygons = load_segmentations_from_txt(segmentation_file)
        
        # Initialize list to store segmentations and attributes
        all_attributes = []
        for polygon in all_polygons:
            points = [Point(p[0], p[1]) for p in polygon]
            mec = welzl(points)  # Compute the minimum enclosing circle
            diameter = mec.R * 2  # Get the diameter of the circle
            all_attributes.append({'diameter': diameter})
        
        # Create Fift   yOne polylines
        segmentations = []

        for polygon, diameter in zip(all_polygons, all_attributes):
            normalized_points = [[(x / 640.0, y / 640.0) for x, y in polygon]]
            
            segmentation = fo.Polyline(
                points=normalized_points,
                closed=True,
                filled=True,
                diameter=diameter['diameter']
            )
            segmentations.append(segmentation)

        
        # Add image and segmentation to FiftyOne dataset
        sample = fo.Sample(filepath=image_path)
        sample["segmentations"] = fol.Polylines(polylines=segmentations)

        dataset.add_sample(sample)

# Save dataset and launch FiftyOne app
dataset.save()
session = fo.launch_app(dataset, port=5153)



# body main

In [1]:
import fiftyone as fo
import os
from data_loader import load_data, create_dataset, process_images
from utils import calculate_euclidean_distance, calculate_real_width, extract_identifier_from_gt
from data_loader_body import load_data, create_dataset, process_images
import MEC
import skeletonization

import sys
from importlib import reload

# Reload the necessary modules
reload(sys.modules['data_loader_body'])
reload(sys.modules['utils'])
reload(sys.modules['MEC'])
reload(sys.modules['skeletonization'])

# Paths to data files
filtered_data_file_path = r"C:\Users\gbo10\Videos\research\counting_research_algorithms\src\measurement\ImageJ\final_full_statistics_with_prawn_ids_and_uncertainty - Copy.xlsx"
metadata_file_path = r"C:\Users\gbo10\OneDrive\research\thesis and paper\test images.xlsx"
car_image_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\car'
car_prediction_folder_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\car_resized"

left_image_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left'
left_prediction_folder_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\left_resized"

right_image_path = r'C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\right'
right_prediction_folder_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\images used for imageJ\check\stabilized\shai\measurements/1\carapace\right_resized"

# Load data and metadata
filtered_df, metadata_df = load_data(filtered_data_file_path, metadata_file_path)
# Create the FiftyOne dataset
molt_image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted"
molt_prediction=r'C:\Users\gbo10\OneDrive\measurement_paper_images\molt\molt-19-9\unditorted_resized'


dataset = create_dataset()


for folder_path, prediction_folder_path in [(car_image_path, car_prediction_folder_path), (left_image_path, left_prediction_folder_path), (right_image_path, right_prediction_folder_path)]:
    image_paths = [os.path.join(folder_path, image) for image in os.listdir(folder_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]
    prediction_paths_text = [os.path.join(prediction_folder_path, txt) for txt in os.listdir(prediction_folder_path) if txt.endswith('.txt')]

    # Process images and predictions using segmentations and bounding boxes
    pond_tag=folder_path.split('/')[-1]

    process_images(image_paths, prediction_folder_path, filtered_df, metadata_df, dataset,pond_tag=pond_tag)

# Start the FiftyOne session
session = fo.launch_app(dataset, port=5153)


 57%|█████▋    | 17/30 [00:01<00:00, 15.36it/s]

No segmentation file found for undistorted_GX010175_215_2644.jpg_gamma


100%|██████████| 30/30 [00:02<00:00, 11.50it/s]


Processed and saved the updated filtered data.


100%|██████████| 5/5 [00:00<00:00,  5.56it/s]


Processed and saved the updated filtered data.


100%|██████████| 37/37 [00:07<00:00,  4.99it/s]


Processed and saved the updated filtered data.

Could not connect session, trying again in 10 seconds



RuntimeError: Client is not connected